<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A5_0_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Automatische Textvergleiche (klassisch) - Teil 1

in den Abschnitten 1 bis 4 haben wir ausschließlich Bilddaten mit unterschiedlichen Methoden des maschniellen Lernens verarbeitet. Jetzt widmen wir uns dem Bereich der Textdaten, bei dem es ganz andere Herausforderungen zu meistern gilt, jedoch ebenso verblüffende Ergebnisse unter Anwendung aktueller Verfahren und hinreichend viel Rechenleistung erreicht werden können.

Grundsätzlich verbrauchen Textdaten wesentlich weniger Datenplatz als Bilder. Zum Vergleich: Ein einzelnes, unkomprimiertes HD-Bild (1920x1080 Pixel, 24bit Farbtiefe) verbraucht mit ca. 6MB mehr Speicher als der gesamte Bibeltext ([ca. 4MB im ANSI/ASCII-Format](https://de.wikipedia.org/wiki/Datenmenge)). Auch die komprimierten Bilder einer herkömmlichen 12MP-Kamera können je nach Motiv trotz Kompression auf über 4MB Speicherbedarf pro Einzelaufnahme kommen.

Letztlich ist der geringe Speicherbedarf von Text nicht weiter verwunderlich, handelt es sich bei den Schriftzeichen, die unsere Sprache kodieren, bereits um einen verschachtelten Code mit einer entsprechend vergleichsweise hohen Informationsdichte. Und anders als bei der uns zu einem Großteil angeborenen Fähigkeit Bilder interpretieren zu können benötigen wir viele Jahre intensiven Trainings, um auch komplizierte Texte, die über Fachtermini, Mehrsilbenwortungetümer oder auch Verzweigungen, die auch dieses Beispiel eines viel zu sehr in die Länge gezogenen Satzes beinhaltet, verfügen, flüssig lesen zu können.

In diesem Notebook werden wir uns zunächst mit einfachen Methoden zur Berechnung von Ähnlichkeitsmaßen zwischen Texten beschäftigen.

Das Verfahren ist wie gehabt:
- Erstellen Sie eine Kopie dieses Notebooks in ihrem Google Drive (vorgeschlagene Umbenennung: "A5_Teil1 - Vorname, Nachname")
- Editieren Sie die Text- und Codezellen.
- Schicken Sie uns einen Freigabelink zum Kommentieren Ihres Notebooks.

## 5.0 Numerische Repräsentation von Text und Ähnlichkeitsmaße

Nach ein paar hilfreichen Imports erstellen wir uns zunächst einen Datensatz. Diesen gilt es dann in eine geeignete numerische Repräsentation zu wandeln, um darauf basierend im Anschluss unterschiedlich aussagekräftige Ähnlichkeitsmaße zwischen den Beispieltexten des Datensatzes zu berechnen.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nltk  # natural language toolkit
import gensim
from pprint import pprint  # pretty print

### Ein einfacher Datensatz

Wir starten mit einem sehr übersichtlichen Datensatz, der aus lediglich vier kurzen Texten besteht. Es handelt sich dabei um Kurzbeschreibungen der Städte Hannover, Braunschweig, München und Nürnberg - jeweils wenige der ersten Sätze der zugehörigen Wikipedia-Seiten.

---

**Anmerkungen**

- Im Kontext der Textdatenverarbeitung wird eine funktionale (z.B. thematische) Texteinheit als Dokument bezeichnet.
- Für die Definition von Zeichenketten (also dem Datentyp `string`), die über mehrere Zeilen reichen, werden in Python wahlweise drei Anführungszeichen (`"""bla"""`) oder drei Hochkommata (`'''blub'''`) verwendet.

In [ ]:
doc1 = """Hannover ist die Hauptstadt des Landes Niedersachsen. Der 
am Südrand des Norddeutschen Tieflandes an der Leine gelegene Ort wurde 1150 
erstmals erwähnt und erhielt 1241 das Stadtrecht. Hannover war ab 1636 welfische 
Residenzstadt, ab 1692 Residenz Kurhannovers, ab 1814 Hauptstadt des Königreichs Hannover, 
nach dessen Annexion durch Preußen ab 1866 Provinzhauptstadt der Provinz Hannover, 
nach Auflösung Preußens im August 1946 Hauptstadt des Landes Hannover und nach 
dessen Fusion mit den Freistaaten Braunschweig, Oldenburg und Schaumburg-Lippe 
im November 1946 niedersächsische Landeshauptstadt. Seit 1875 Großstadt, 
zählt Hannover heute mit 538.068 Einwohnern (Ende 2018) zu den 15 
einwohnerreichsten Städten Deutschlands."""

doc2 = """Braunschweig ist eine Großstadt im Südosten des Landes Niedersachsen. 
Mit 248.292 Einwohnern (Stand 31. Dezember 2018) ist sie nach Hannover die 
zweitgrößte Stadt Niedersachsens. Die kreisfreie Stadt bildet mit den Städten 
Salzgitter und Wolfsburg eine Regiopolregion und eines der neun Oberzentren des 
Bundeslandes. Sie ist Teil der im Jahr 2005 gegründeten Metropolregion 
Hannover-Braunschweig-Göttingen-Wolfsburg. Im Großraum Braunschweig wohnen rund 
eine Million Menschen."""

doc3 = """München ist die Landeshauptstadt des Freistaates Bayern. Sie ist mit 
etwa 1,5 Millionen Einwohnern die einwohnerstärkste Stadt Bayerns und (nach Berlin
und Hamburg) die nach Einwohnern drittgrößte Gemeinde Deutschlands. Sie bildet 
das Zentrum der Metropolregion München (rund 6 Millionen Einwohner) und der 
Planungsregion München (rund 2,9 Millionen Einwohner). München ist eine kreisfreie Stadt. 
Sie ist Verwaltungssitz des die Stadt umgebenden Landkreises München mit dessen Landratsamt
sowie des bayerischen Bezirks Oberbayern und des Regierungsbezirks Oberbayern."""

doc4 = """Nürnberg ist eine fränkische kreisfreie Großstadt im Regierungsbezirk 
Mittelfranken des Freistaats Bayern. Nürnberg ist mit rund 520.000 Einwohnern nach 
München die größte Stadt Bayerns, weist dabei ein deutliches Wachstum auf und gehört 
damit zu den 15 größten Städten Deutschlands. Zusammen mit den direkten Nachbarstädten 
Fürth, Erlangen und Schwabach bildet Nürnberg mit rund 800.000 Einwohnern eine der 
drei Metropolen in Bayern. Gemeinsam mit ihrem Umland bilden diese Städte den 
Ballungsraum Nürnberg, mit über 1,3 Millionen Menschen und das wirtschaftliche 
und kulturelle Zentrum der knapp 3,6 Millionen Einwohner umfassenden Europäischen 
Metropolregion Nürnberg, eine der 11 Metropolregionen in Deutschland."""

Unsere vier Dokumente können wir in einen geeigneten Container-Datentyp packen, z.B. in eine Liste. Eine solche Sammlung von Dokumenten wird im Fachjargon als Korpus bezeichnet.

---

**Anmerkungen**

Nur zur Erinnerung und Übersicht: Die drei am häufigsten verwendeten Container-Datentypen in Python sind `tuple` (runde Klammern), `list` [eckige Klammern] und `dict` {geschweifte Klammern}.

In [ ]:
corpus = [doc1, doc2, doc3, doc4]

### Numerische Repräsentation der Texte

Da wir die Textdaten automatisiert durchsuchen, vergleichen oder andersartig analysieren wollen, ist es sehr hilfreich, wenn wir eine adäquate numerische Repräsentation verwenden. Diese kann je nach den Problemstellungen, die gelöst werden sollen, variieren.

In einem ersten Schritt zur numerischen Darstellung der Texte werden diese in atomare Bestandteile, die auch als **Tokens** bezeichnet werden, zerlegt. Bereits hier gibt es unterschiedliche Möglichkeiten für diese sogenannte "Tokenisierung". Eine typische und naheliegende Definition ist es, dass ein Token einem Wort entspricht.

---

**Anmerkungen**

Wir verwenden hier zum Tokenisieren den Standard-Tokenizer der Open-Source-Bibliothek [`Gensim`](https://en.wikipedia.org/wiki/Gensim).

In [ ]:
def tokenizer(text):
    tokens = list(gensim.utils.simple_tokenize(text))
    return tokens
# Hier kommt eine sogenannte list comprehension zum Einsatz. Das ist eine for-Schleife,
# die in eckigen Klammern steht und als Ergebnis eine Liste liefert.
corpus_tokenized = [tokenizer(document) for document in corpus]

# Beispielhafte Ausgabe der tokenisierten Version des Hannover-Textes
pprint(corpus_tokenized[0], compact=True) 

Wie wir sehen beinhalten die Tokens keine Satzzeichen wie Kommas, Punkte oder Bindestriche. Auch Zahlen werden bei der Tokenisierung nicht beachtet.

### Bag of words

Als nächstes ordnen wir im Zuge der numerischen Repräsentation jedem Token eine eineindeutige Zahl zu ...

In [ ]:
dictionary = gensim.corpora.Dictionary(corpus_tokenized)
pprint(dictionary.token2id)

... und erhalten somit als numerische Darstellung unserer tokenisierten Texte eine Liste von Zahlen:

In [ ]:
pprint(dictionary.doc2idx(corpus_tokenized[0]), compact=True)

Alternativ können wir uns auch eine sogenannte *Bag of Words* (bow) ausgeben lassen. Dabei wird der numerische Schlüssel eines jeden Tokens zusammen mit seiner Auftrittshäufigkeit innerhalb des Korpus ausgegeben.

In [ ]:
# bow am Beispiel des Hannover-Textes
pprint(dictionary.doc2bow(corpus_tokenized[0]), compact=True)

Wir können uns somit auch anschauen, wie häufig ein jedes Token in den vier Dokumenten auftritt.

In [ ]:
# Ganzer Korpus numerisiert
corpus_numeric = [dictionary.doc2bow(token_list) for token_list in corpus_tokenized]
# Kompakte Version auffüllen mit Nullen - daher muss die Methode wissen, wieviele Spalten existieren
gensim.matutils.corpus2dense(corpus_numeric, num_terms=len(dictionary))

### Automatischer Textvergleich

Mit Hilfe dieser Repräsentation unserer Texte können wir nun effizient mathematische Operationen ausführen lassen, z.B. Ähnlichkeiten zwischen den vier Texten berechnen.

Bei der Ähnlichkeitsberechnung wird im wesentlichen verglichen, wie stark sich das Vokabular von zwei Dokumenten überschneidet.

In [ ]:
similarity_index = gensim.similarities.SparseMatrixSimilarity(corpus_numeric, num_features=len(dictionary))
all2all_similarity = similarity_index[corpus_numeric]

In [ ]:
plt.imshow(all2all_similarity, cmap='hot')
plt.xticks(range(4), ['H', 'Bs', 'M', 'N'])
plt.yticks(range(4), ['H', 'Bs', 'M', 'N'])
pax = plt.colorbar()
pax.set_label('Ähnlichkeitsmaß')
_ = plt.title('Vergleich des Vokabulars')

Schauen wir uns an, welche der Tokens (sprich welche Wörter) am stärksten zu dem hier verwendeten Ähnlichkeitsmaß zwischen den ersten beiden Dokumenten (Hannover und Braunschweig) beitragen. Hierfür definieren wir uns zunächst die Funktion `top_similarity_contribution()`.

In [ ]:
#@title
#@markdown Bitte ausführen: Definition von **top_similarity_contributions**

def top_similarity_contributions(doc_numeric1, doc_numeric2, n_best=7):
    
    d1 = {token_id: count for token_id, count in doc_numeric1}
    d2 = {token_id: count for token_id, count in doc_numeric2}
    both = {}
    for token_id in d1:
        if token_id in d2:
            counts1 = d1[token_id]
            counts2 = d2[token_id]
            both[token_id] = (counts1, counts2, counts1*counts2)
    best = sorted(both.items(), key=lambda x : x[1][2], reverse=True)
    print('Stärkste Beiträge zur Ähnlichkeit:')
    print('----------------------------------')
    for ix in range(min(n_best, len(both))):
        token_id, data = best[ix]
        print(f'{dictionary[token_id]} (id: {token_id}) hat Score {data[2]:.3f}={data[0]:.2f}*{data[1]:.2f}')

Diese können wir jetzt für sämtliche Dokument-Paarungen unseres Korpus verwenden. Für die Paarung Hannover-Braunschweig liefert sie folgendes Ergebnis:

In [ ]:
top_similarity_contributions(corpus_numeric[0], corpus_numeric[1])

Die Texte über Braunschweig und München ähneln sich dagegen wie folgt:

In [ ]:
top_similarity_contributions(corpus_numeric[1], corpus_numeric[2])

### Stopwords

Wie wir sehen, beruht die Ähnlichkeit der Texte (bzw. der Dokumente) auf dem gemeinsamen Vorkommen von Wörten wie "der", "des", "und" u.ä. Jedoch können wir davon ausgehen, dass diese Wörter keinen Beitrag zum eigenltichen Inhalt liefern. Diese Art von Wörtern wir auch als Stopwörter genannt. Lassen Sie uns deshalb die Analyse ohne Stopwörter wiederholen.

Die Bibliothek `nltk` liefert eine Zusammenstellung der Stoppwörter für verschiedenen Sprachen inkl. Deutsch. Hierauf basierend können wir uns eine Hilfsfunktion definieren, die uns angibt, ob es sich bei einem Wort um ein gelistetes Stoppwort handelt.

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('german')
def is_not_stopword(word):
    return word not in stopwords
# Ein kurzer Blick auf alle gelisteten Stopwörter
pprint(stopwords, compact=True)

Diese Funktion können wir jetzt als Filterfunktion verwenden, um die Stopwörter aus unseren Dokumenten heraus zu filtern. Anschließend können wir erneut die Ähnlichkeitsmaße berechnen lassen - dieses Mal entsprechend anhand der gefilterten Dokumente.

In [ ]:
# Wir filtern die Stopwörter aus unseren Dokumente 
# und wandeln die restlichen Tokens wieder in Zahlen um
corpus_numeric_filtered = [dictionary.doc2bow(filter(is_not_stopword, token_list))
                          for token_list in corpus_tokenized]

# Jetzt berechnen wir wieder die Ähnlichkeit aller Dokumentenpaarungen.
similarity_index2 = gensim.similarities.SparseMatrixSimilarity(corpus_numeric_filtered, num_features=len(dictionary))
all2all_similarity_no_stopwords = similarity_index2[corpus_numeric_filtered]

In [ ]:
plt.imshow(all2all_similarity_no_stopwords, cmap='hot')
plt.xticks(range(4), ['H', 'Bs', 'M', 'N'])
plt.yticks(range(4), ['H', 'Bs', 'M', 'N'])
pax = plt.colorbar()
pax.set_label('Ähnlichkeitsmaß')
_ = plt.title('Vergleich ohne Stopwörter')

Erneut der Blick auf die Wörter, die zur Ähnlichkeit zwischen Hannover und Braunschweig sowie zwischen Braunschweig und München beitragen:

In [ ]:
print("HANNOVER - BRAUNSCHWEIG:")
top_similarity_contributions(corpus_numeric_filtered[0], corpus_numeric_filtered[1])
print("\nBRAUNSCHWEIG - MÜNCHEN:")
top_similarity_contributions(corpus_numeric_filtered[1], corpus_numeric_filtered[2])

Wir sehen nun, dass zwar nicht mehr allgemeine Stopwörter einen hohen Einfluss auf das Ähnlichkeitsmaß haben, aber dass es weiterhin korpus-spezifische Wörter gibt ("Einwohner", "Städten", ...) die wir nicht als inhaltsspezifisch bewerten würden.  

### TFIDF
Eine Möglichkeit, den Einfluss solcher eigentlich unspezifischen Wörter zu verringern und damit die Aussagekraft der Vergleiche zu erhöhen, stellt das TFIDF-Verfahren (Term Frequency - Inverse Document Frequency) dar. Hierbei wird der Informationsgehalt der Wörter nach ihrem Vorkommen in allen Dokumenten des gesamten Korpus gewichtet: Ein Wort, das auch in vielen anderen Dokumenten vorkommt, sagt nur wenig über die Ähnlichkeit von Texten. Ein Wort, das nur in wenigen Dokumenten vorkommt, ist ein stärkerer Indikator dafür, dass sich deren Texte thematisch ähneln. 

In [ ]:
tfidf = gensim.models.TfidfModel(corpus_numeric)
print(corpus_numeric[0])
print(tfidf[corpus_numeric[0]])

In [ ]:
similarity_index_tfidf = gensim.similarities.SparseMatrixSimilarity(tfidf[corpus_numeric], num_features=len(dictionary))
all2all_similarity_tfidf = similarity_index_tfidf[tfidf[corpus_numeric]]

In [ ]:
plt.imshow(all2all_similarity_tfidf, cmap='hot')
_ = plt.xticks(range(4), ['H', 'Bs', 'N', 'M'])
_ = plt.yticks(range(4), ['H', 'Bs', 'N', 'M'])
pax = plt.colorbar()
pax.set_label('Ähnlichkeitsmaß')
_ = plt.title('Vergleich mittels TFIDF')

In [ ]:
top_similarity_contributions(tfidf[corpus_numeric[0]], tfidf[corpus_numeric[1]])

### 5.0.0 Erläutern Sie die Begriffe *Token und Tokenisierung*.

<bitte ausfüllen>

### 5.0.1 Erläutern Sie den Begriff *BoW* (Bag of Words)

<bitte ausfüllen>

### 5.0.2 Versuchen Sie mit eigenen Worten das Prinzip des Verfahrens *TF-IDF* (Term Frequency - Inverse Document Frequency) zu erklären.

<bitte ausfüllen>

### 5.0.3 Artifizielle Ähnlichkeit

Denken sie sich einen kurzen Text aus, der im TF-IDF-Verahren ein möglichst hohes Ähnlichkeitsmaß zum Hannover-Text aufweist, jedoch im direkten, "manuellen" Verleich nicht. Tragen Sie den Text in die Variable `new_text` ein.

In [ ]:
new_text = '''bitte hier Ihren Text eintragen'''

tokenized_text = tokenizer(new_text)
numeric_text = dictionary.doc2bow(tokenized_text)

similarity_basic = similarity_index[numeric_text]
similarity_tfidf = similarity_index_tfidf[numeric_text]

plt.imshow(np.vstack([similarity_basic, similarity_tfidf]), cmap='hot', vmin=0)
plt.yticks([0,1], labels=['index', 'index_tfidf'])
plt.xticks([0,1,2,3], labels=['H', 'Bs', 'M', 'N'])
cax = plt.colorbar()
cax.set_label('Ähnlichkeitsmaß')

Erläutern Sie das Ergebnis.

<bitte ausfüllen>

### 5.0.4 Weitere Ähnlichkeiten berechnen

Definieren Sie sich drei weitere Dokumente mit jeweils mindestens 500 Wörtern. Wählen Sie dabei zwei thematisch verwandte Textquellen (z.B. zwei Ausschnitte aus dem gleichen Roman) und einen Text, der aus einer völlig anderen Quelle stammt.

In [ ]:
# Dokumente definieren
my_doc1 = ...
...

Verfahren Sie nun mit Ihren drei Dokumenten analog zu den Schritten aus Abschnitt 5.0:
- Numerische Repräsentation der Texte
- Bag of words
- Stopwords
- TF/IDF

---

**Anmerkungen**

- Der Übersichtlichkeit halber können Sie diese Aufgabe auch in einem gesonderten Notebook bearbeiten.
- Es kann sehr hilfreich sein, jeden Schritt bzw. kleine Gruppen von Anweisungen zunächst in einem Kommentar zu beschreiben.

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)